# Uploading the Datasets to COLAB VM

In [0]:
import io
from os import path
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
from googleapiclient.discovery import build


In [0]:
# Specify the GDrive file-ID and filename to save the file. 
file_id = '1zsUzSIuZXbRkaM7T64EUtvV6OHWgAZKw'
file_name = 'ferplus.csv'

# Authentificate Google Drive SDK.
auth.authenticate_user()

# Load the drive service.
drive_service = build('drive', 'v3')

# If file is not present, download.
if not path.isfile(file_name):

  # Build download request and media downloader.
  request = drive_service.files().get_media(fileId=file_id)
  output_file = io.FileIO(file_name, mode='wb')
  downloader = MediaIoBaseDownload(output_file, request)
  done = False

  # Download chunks.
  while done is False:
      status, done = downloader.next_chunk()
      print('Download %d%%.' % int(status.progress() * 100))

  print('Download Complete!')

In [8]:
# Check if dataset is there.
!ls -lh

total 288M
drwxr-xr-x 1 root root 4.0K Mar 20 10:19 datalab
-rw-r--r-- 1 root root 288M Mar 20 10:19 ferplus.csv


# Data Preprocessing

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

tf.__version__

'1.6.0'

In [10]:
# read csv with pandas
ferplus = pd.read_csv('ferplus.csv')
ferplus

,Usage,pixels,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,4,0,0,1,3,2,0,0,0,0
1,Training,151 150 147 155 148 133 111 140 170 174 182 15...,6,0,1,1,0,0,0,0,2,0
2,Training,231 212 156 164 174 138 161 173 182 200 106 38...,5,0,0,3,1,0,0,0,1,0
3,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,4,0,0,4,1,0,0,0,1,0
4,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,9,0,0,1,0,0,0,0,0,0
5,Training,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,6,0,0,1,0,0,1,1,1,0
6,Training,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,2,0,0,8,0,0,0,0,0,0
7,Training,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,0,10,0,0,0,0,0,0,0,0
8,Training,85 84 90 121 101 102 133 153 153 169 177 189 1...,0,10,0,0,0,0,0,0,0,0
9,Training,255 254 255 254 254 179 122 107 95 124 149 150...,0,0,6,0,0,0,4,0,0,0


In [11]:
# Extract and cast pixel intensities.
ferplus.pixels = ferplus.pixels.str.split()
ferplus.pixels = ferplus.pixels.map(lambda p: pd.to_numeric(p, downcast='float'))

# Check type of pixel value.
ferplus.pixels[0].dtype

dtype('float32')

In [0]:
# Filter out noface class.
ferplus = ferplus.query('NF==0')

In [13]:
# Get argmax of class distribution to use as label for each image.
labels = ferplus[['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']]
maxlabels = labels.idxmax(axis=1).map(labels.columns.get_loc)
ferplus.insert(loc=12, column='maxlabel', value=maxlabels)
ferplus

,Usage,pixels,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF,maxlabel
0,Training,"[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63....",4,0,0,1,3,2,0,0,0,0,6
1,Training,"[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 111...",6,0,1,1,0,0,0,0,2,0,6
2,Training,"[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 161...",5,0,0,3,1,0,0,0,1,0,6
3,Training,"[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20....",4,0,0,4,1,0,0,0,1,0,4
4,Training,"[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9,0,0,1,0,0,0,0,0,0,6
5,Training,"[55.0, 55.0, 55.0, 55.0, 55.0, 54.0, 60.0, 68....",6,0,0,1,0,0,1,1,1,0,6
6,Training,"[20.0, 17.0, 19.0, 21.0, 25.0, 38.0, 42.0, 42....",2,0,0,8,0,0,0,0,0,0,4
7,Training,"[77.0, 78.0, 79.0, 79.0, 78.0, 75.0, 60.0, 55....",0,10,0,0,0,0,0,0,0,0,3
8,Training,"[85.0, 84.0, 90.0, 121.0, 101.0, 102.0, 133.0,...",0,10,0,0,0,0,0,0,0,0,3
9,Training,"[255.0, 254.0, 255.0, 254.0, 254.0, 179.0, 122...",0,0,6,0,0,0,4,0,0,0,5


In [0]:
# Split train, test and validation set.
train = ferplus.loc[ferplus['Usage'] == 'Training']
valid = ferplus.loc[ferplus['Usage'] == 'PublicTest']
test = ferplus.loc[ferplus['Usage'] == 'PrivateTest']

In [0]:
x_test = np.array(test['pixels'].values.tolist())
x_train = np.array(train['pixels'].values.tolist())
x_valid = np.array(valid['pixels'].values.tolist())

y_test = test['maxlabel'].values
y_train = train['maxlabel'].values
y_valid = valid['maxlabel'].values


In [16]:
labels.shape

(35538, 7)

## Network and Training Parameters

In [0]:
# images are 48x48 pixels
img_size = 48
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

# Classes and class labels
num_classes = 7
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# probability to drop a unit (prevent overfitting)
dropout = 0.3

# training parameters
learning_rate = 0.001
max_steps = 1500
# num_epochs = 10
batch_size = 128

## Model Architecture

In [0]:
# Create the neural network
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']

        # The data input is a 1-D vector of 2304 features (48*48 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # 4D Input-Tensor: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, img_size, img_size, 1])

        tf.summary.image("img", x)

        # Convolution Layers with 32 filters and a kernel size of 3.
        conv1 = tf.layers.conv2d(x, 32, 3, padding='same', activation=tf.nn.relu)
        conv1 = tf.layers.conv2d(conv1, 32, 3, padding='same', activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layers with 64 filters and a kernel size of 3.
        conv2 = tf.layers.conv2d(conv1, 64, 3, padding='same', activation=tf.nn.relu)
        conv2 = tf.layers.conv2d(conv2, 64, 3, padding='same', activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2.
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        
        # Convolution Layer with 128 filters and a kernel size of 3
        conv3 = tf.layers.conv2d(conv2, 128, 3, padding='same', activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(conv3, 128, 3, padding='same', activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(conv3, 128, 3, padding='same', activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2.
        conv3 = tf.layers.max_pooling2d(conv3, 2, 2)

        # Convolution Layers with 256 filters and a kernel size of 3.
        conv4 = tf.layers.conv2d(conv3, 256, 3, padding='same', activation=tf.nn.relu)
        conv4 = tf.layers.conv2d(conv4, 256, 3, padding='same', activation=tf.nn.relu)
        conv4 = tf.layers.conv2d(conv4, 256, 3, padding='same', activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2.
        conv4 = tf.layers.max_pooling2d(conv4, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer.
        fc = tf.contrib.layers.flatten(conv4)

        # Fully connected layer.
        fc = tf.layers.dense(fc, 1024)
        # Apply Dropout (only when is_training is True).
        fc = tf.layers.dropout(fc, rate=dropout, training=is_training)

        # Output layer, class prediction.
        out = tf.layers.dense(fc, n_classes)

    return out

In [0]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode,
          predictions=pred_classes,
          export_outputs={'classes': tf.estimator.export.PredictOutput(pred_classes)}) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [38]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn=model_fn, model_dir='model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1d5cad8550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [0]:
!rm -Rf model

In [26]:
# Define the input function for training.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_train}, y=y_train,
    batch_size=batch_size, num_epochs=None, shuffle=True)

# Define the input function for validation during training.
valid_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_valid}, y=y_valid,
    batch_size=batch_size, num_epochs=None, shuffle=False)

# Specify training operations.
train_spec = tf.estimator.TrainSpec(
    input_fn = train_input_fn,
    max_steps = max_steps
)

# Specify evaluation operations.
eval_spec = tf.estimator.EvalSpec(
    input_fn = valid_input_fn,
    throttle_secs=30,
    start_delay_secs=30,
)

# Train the Model and evaluate periodically
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 30 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into model/model.ckpt.
INFO:tensorflow:loss = 2.6859374, step = 1
INFO:tensorflow:global_step/sec: 9.97444
INFO:tensorflow:loss = 1.6198416, step = 101 (10.032 sec)
INFO:tensorflow:global_step/sec: 10.1456
INFO:tensorflow:loss = 1.5036383, step = 201 (9.852 sec)
INFO:tensorflow:Saving checkpoints for 289 into model/model.ckpt.
INFO:tensorflow:Loss for final step: 1.4219484.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-19-14:28:25
I

INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-03-19-14:31:05
INFO:tensorflow:Saving dict for global step 1500: accuracy = 0.7202344, global_step = 1500, loss = 0.8189332


## Evaluation

In [39]:
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': x_test}, y=y_test,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-19-15:38:45
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-1500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-19-15:38:47
INFO:tensorflow:Saving dict for global step 1500: accuracy = 0.6977538, global_step = 1500, loss = 0.873001


{'accuracy': 0.6977538, 'global_step': 1500, 'loss': 0.873001}

# Exporting as tf.SavedModel

In [40]:
# Export the model as a SavedModel for production use
feature_spec = {'images': tf.placeholder(dtype=tf.float32, shape=[None, img_size * img_size])}
serving_input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(feature_spec)

model.export_savedmodel(
    export_dir_base='saved_models/ferplus',
    serving_input_receiver_fn=serving_input_fn,
    as_text=True)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['classes', 'serving_default']
INFO:tensorflow:Restoring parameters from model/model.ckpt-1500
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"saved_models/ferplus/temp-b'1521473931'/saved_model.pbtxt"


b'saved_models/ferplus/1521473931'

## Download SavedModel to Gdrive

In [52]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('saved_models/ferplus/1521473931/saved_model.pbtxt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))


File ID: 19ysx5knym8ZL7PmHeQgSx3JC5RxdslI4
